In [53]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# __author__ = 'Prakhar MISRA'
# Created 08/14/2019
# Last edit 08/14/2019

#

# Purpose: to get non interpolated values of R for the rainy seasons (months 6-7)

import pandas as pd
import numpy as np
import os
import glob 

currdir = os.getcwd()

In [66]:

# original directory where R for each city is present
R_city_directory = os.path.join(currdir, "lib", "observedAQImasumodII")

# file used in stan simulation
AYpath = os.path.join(currdir, "my_bayesian", "HBM_city_tier_20180116", "report20180212_IDW2_20190814")
AYfile = os.path.join(AYpath, "allAY20180212IDW2setclean.csv")


In [55]:
# first get the R in months 6,7 fromt he original file


listdf =[]

# open each csv and get the R for only selected months
for file in glob.glob(os.path.join(R_city_directory, '*' + ".csv")):
    
    #file = glob.glob(os.path.join(R_city_directory, '*' + ".csv"))[0]
    pd_R = pd.read_csv(file, header = 0)
    pd_R["date"] = pd.to_datetime(pd_R["date"], format = "%Y%m")
    pd_R = pd_R[(pd_R["date"].dt.year <= 2015)]
    pd_filter = pd_R[(pd_R["date"].dt.month == 6) | (pd_R["date"].dt.month == 7)]
    #pd_filter = pd_R[(pd_R["date"].dt.month <= 12)]
    
    # applying humidity correction
    pd_filter["R"] = pd_filter["R"]*11/14
    
    listdf.append(pd_filter[["city", "R", "date"]])
    
df_allcityrainyR = pd.concat(listdf, axis = 0)

df_allcityrainyR.to_csv(os.path.join(R_city_directory, "rainy_month", "df_month_6_7_R.csv"))

C:\Users\Prakhar\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [56]:
# now replace the R values in the getAY (i.e. thje file inputted to the stan) for the months 6, 7
# create first multi index


# replace the R corectly for each city
dfAY = pd.read_csv(AYfile, header=0)
dfAY["date"] = pd.to_datetime(dfAY["date"], format = "%m/%d/%Y")
# set the multi index
dfAY.index = pd.MultiIndex.from_arrays(dfAY[['citychk_loc', 'date']].values.T, names=['idx1', 'idx2'])


# also open R for the months 6, 7
df_rainR = pd.read_csv(os.path.join(R_city_directory, "rainy_month", "df_month_6_7_R.csv"), header=0)
df_rainR["date"] = pd.to_datetime(df_rainR["date"], format = "%Y-%m-%d")
df_rainR["Rrain"] = df_rainR["R"]
# set the multi index
df_rainR.index = pd.MultiIndex.from_arrays(df_rainR[['city', 'date']].values.T, names=['idxR1', 'idxR2'])
df_rainR.drop(columns = ["city",  "date"], inplace = True)




In [57]:
# and merge on the basis of multiindex

# need to give name to index level
dfAY.index.levels[0].name = "idx1"
dfAY.index.levels[1].name = "idx2"
df_rainR.index.levels[0].name = "idx1"
df_rainR.index.levels[1].name = "idx2"

# now merge
df_merge = pd.merge(dfAY, df_rainR, how='outer', on=None, left_on=None, right_on=None,
         left_index=True, right_index=True, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

df_merge["R"] = df_merge[["R_x", "R_y"]].min(axis =1)

# save this and clean this
df_merge.sort_values(by = ["TierID", "cityID2", "citychk_loc", "date"]).to_csv(os.path.join(AYpath, "allAY20180212IDW2setclean_rainyR.csv"))

In [74]:
# only keep the 2001 and 2011 for training
df_AYrainR = pd.read_csv(os.path.join(AYpath, "allAY20180212IDW2setclean_rainyR.csv"), header=0)
df_AYrainR["date"] = pd.to_datetime(df_AYrainR["date"], format = "%m/%d/%Y")
pd_filter = df_AYrainR[(df_AYrainR["date"].dt.year == 2001) | (df_AYrainR["date"].dt.year == 2011)]

# sacve
df_AYrainR.sort_values(by = ["TierID", "cityID2", "citychk_loc", "date"]).to_csv(os.path.join(AYpath, "allAY20180212IDW2setclean_rainyR20012011.csv"))